# Модель классификации комментариев

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression

import re

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
from catboost import CatBoostClassifier
#import lightgbm as ltb
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikvasily/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikvasily/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nikvasily/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


При подготовке комментариев к обучению проделаем следующие этапы:
1. Очистка комментариев от системного и лишнего текста
2. Лемматизация комментариев
3. Создание признаков из коментариев по значениям TF-IDF

Но для начала ознакомимся с данными:

In [2]:
pd.set_option('display.max_colwidth', -1)
data = pd.read_csv("toxic_comments.csv")

print(data.info())
print()

print(data['text'].head(30))
print()

print(data['toxic'].head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159571 non-null  int64 
 1   text        159571 non-null  object
 2   toxic       159571 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.7+ MB
None

0     Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27                                                                                                                                                                                                                                                                                                                                                                      

Просмотрев первые 10 постов, замечаем следующие недостатки для устранения перед обучением:
1. Переносы строк "\n" нам явно не помошники, нужно будет избавиться от них переводом в юникод +
2. Встречаются подобные значения в конце постов: "89.205.38.27". Похоже на номер пользователя или IP-адрес? Пользы для оценки токсичности они явно не несут, уберутся оставив только буквы. +
3. Встречаются даты в конце постов: "21:51, January 11, 2016 (UTC)". Дата нам тоже явно не поможет.. придумаем для нее специальный шаблон. +
4. Встречаются подобные значения (talk) +
5. Тексты могут быть внесены в кавычки, от них также избавимся оставив только буквы. +
6. Есть сноски на статьи, но их названия также могут помочь в определении токсичности, заменим только _ на пробелы: "Wikipedia:Good_article_nominations#Transport"     
7. "· talk "   " +
8. "Image:Wonju.jpg" - лишние фотографии  "Unspecified source for Image:Wonju.jpg" +
9. "• contribs • " - похоже на смайлики +
10. Ненормативная лексика.. 


В следующем блоке кода написана функция по очистке текста с помощью re.sub в соответствии с выявленными в комментариях "лишними" словосочетаниями, датами и т.п. 

In [3]:
def clear_text(text):
    t = re.sub(r'[\d\d]+[:]+[\d\d]+[,]+[ ]+[0-9a-zA-Z]+[ ]+[0-9a-zA-Z]+[,| ]+', ' ', text.lower())
    t = re.sub(r'[(]+[a-zA-Z0-9• ]+[)]', ' ', t)
    t = re.sub(r'[·•]+[ ]+[a-zA-Z0-9]+[ ]+', ' ', t)
    t = re.sub(r'[Image]+[:]+[a-zA-Z0-9]+[.]+[jpg]+', ' ', t)
    t = re.sub(r'[_·•]', ' ', t)
    t = re.sub(r'[^a-zA-Z ]', ' ', t)   
    return " ".join(t.split())

#проверка
print(data['text'][1])
print()
print(clear_text(data['text'][1]))


D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)

d aww he matches this background colour i m seemingly stuck with thanks


Добавим теперь столбец с очищенными комментариями

In [4]:
#corpus = data['text'].values.astype('U')

data['clear_text'] = data['text'].apply(lambda x: clear_text(x))
#проверка
print(data['clear_text'].head())

0    explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now                                                                                                                                                                                                                                                                                                                                                                 
1    d aww he matches this background colour i m seemingly stuck with thanks                                                                                                                                                                                                                                                                                                                             

Перейдем к лемматизации, применим библиотеку nltk WordNetLemmatizer, лемматизировать будем по отдельным словам:

In [5]:
lemmatizer = WordNetLemmatizer()

def lemmatizing(text):
    word_list = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])


data['lemm_text'] = data['clear_text'].apply(lambda x: lemmatizing(x))

#проверка
print(data.head())

   Unnamed: 0  \
0  0            
1  1            
2  2            
3  3            
4  4            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 text  \
0  Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.3

Лемматизация проведена, пора создавать признаки по TF-IDF

In [6]:

features = data['lemm_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.13, random_state = 123)
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)
#print(features_train.head(5))

corpus = list(features_train)
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words = stop_words)
features_train = count_tf_idf.fit_transform(corpus)



(138826,)
(20745,)
(138826,)
(20745,)


## Обучение

Перейдем к обучению и тестированию модели Логистической регрессии:

In [7]:
%%time
model = LogisticRegression()

#print()
#print(features_train)
model.fit(features_train, target_train)

corpus_test = list(features_test)
features_test = count_tf_idf.transform(corpus_test)

predictions = model.predict(features_test)
print('Значение метрики F1 на тестовой выборке составило:', f1_score(target_test, predictions))

Значение метрики F1 на тестовой выборке составило: 0.7526439482961222
CPU times: user 5.17 s, sys: 6.34 s, total: 11.5 s
Wall time: 3.29 s


Моделью логистической регрессии получилось достичь требуемого значения метрики F1, перейдем теперь к случайному лесу:

In [8]:
%%time

model_rf = RandomForestClassifier(random_state = 12345)
model_rf.fit(features_train, target_train)

rf_predictions = model_rf.predict(features_test)
print('Значение метрики F1 на тестовой выборке составило:', f1_score(target_test, rf_predictions))

Значение метрики F1 на тестовой выборке составило: 0.700797057020233
CPU times: user 4min 43s, sys: 496 ms, total: 4min 44s
Wall time: 4min 44s


Случайный лес не позволил увеличить значение метрики, а скорее слабо себя показал (впервые в проектах), значение метрики более чем на 5% ниже (0.7).
Теперь попробуем CatBoost

In [9]:
features = data['lemm_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.13, random_state = 123)

corpus = list(features_train)

count_tf_idf = TfidfVectorizer(stop_words = stop_words)
features_train = count_tf_idf.fit_transform(corpus)

model_cb = CatBoostClassifier(eval_metric="F1", depth = 6, iterations = 100, random_seed = 12345)
model_cb.fit(features_train, target_train, verbose=20) 


Learning rate set to 0.5
0:	learn: 0.3944136	total: 779ms	remaining: 1m 17s
20:	learn: 0.6757520	total: 7.48s	remaining: 28.1s
40:	learn: 0.7236159	total: 14s	remaining: 20.1s
60:	learn: 0.7488175	total: 20.4s	remaining: 13s
80:	learn: 0.7593499	total: 26.8s	remaining: 6.28s
99:	learn: 0.7735192	total: 33s	remaining: 0us


In [10]:
corpus_test = list(features_test)
features_test = count_tf_idf.transform(corpus_test)

cb_predictions = model_cb.predict(features_test)
print('Значение метрики F1 на тестовой выборке составило:', f1_score(target_test, cb_predictions))

Значение метрики F1 на тестовой выборке составило: 0.7612024957458876


CatBoost даже на легких случайных параметрах (хоть и при 10ти минутном обучении), выдал значение метрики в 0,76. Попробуем еще немного улучшить качество работы логистической регрессии, уж больно быстро она обучается по сравнению с другими моделями. Для этого в TF-IDF Vectorizer попробуем передать не униграммы, а униграммы и биграммы:

In [11]:
#features = data['lemm_text']
#target = data['toxic']

#features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.13, random_state = 123)

#corpus = list(features_train)

#count_tf_idf2 = TfidfVectorizer(stop_words = stop_words, ngram_range=(1,2))
#features_train2 = count_tf_idf2.fit_transform(corpus)

На данном этапе ядро отваливается.. Поскольку все же интересно, дают ли прирост включение биграмм, попробуем из всего датасета взять всего 10000 строк и прогнать на них модели.

In [12]:
comments = data.sample(10000).reset_index(drop=True)

In [13]:
features = comments['lemm_text']
target = comments['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 123)

corpus = list(features_train)
count_tf_idf = TfidfVectorizer(stop_words = stop_words)
features_train = count_tf_idf.fit_transform(corpus)

model = LogisticRegression()

model.fit(features_train, target_train)

corpus_test = list(features_test)
features_test = count_tf_idf.transform(corpus_test)

predictions = model.predict(features_test)
print('Значение метрики F1 на тестовой выборке составило:', f1_score(target_test, predictions))

Значение метрики F1 на тестовой выборке составило: 0.5035460992907801


Для начала на сокращенном датасете оценим значение метрики на униграммах: 
- Значение метрики F1 на тестовой выборке составило: 0.41

Не густо, но само значение нам нужно для сравнения

In [14]:
features = comments['lemm_text']
target = comments['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 123)
corpus = list(features_train)
count_tf_idf = TfidfVectorizer(stop_words = stop_words, ngram_range = (2,2))
features_train = count_tf_idf.fit_transform(corpus)

model = LogisticRegression()

model.fit(features_train, target_train)

corpus_test = list(features_test)
features_test = count_tf_idf.transform(corpus_test)

predictions = model.predict(features_test)
print('Значение метрики F1 на тестовой выборке составило:', f1_score(target_test, predictions))

Значение метрики F1 на тестовой выборке составило: 0.028169014084507043


Добавив к униграммам биграммы, получаем следующий результат:
- Значение метрики F1 на тестовой выборке составило: 0.33

Только с биграммами:
- Значение метрики F1 на тестовой выборке составило: 0.01

униграммы, биграммы и триграммы:
- Значение метрики F1 на тестовой выборке составило: 0.29

еще и 4-ч и 5-ти граммы:
- Значение метрики F1 на тестовой выборке составило: 0.25

Делаем вывод о том, что по крайней мере на сокращенном датасете добавление к униграммам биграмм, триграмм и так далее только снижает качество работы модели Лоистической регрессии. Могу предположить, что биграммы и граммы высшего порядка повторяются существенно реже и только забирают на себя вес коэффициентов логистической регрессии не приводя у улучшению качества модели. Вполне вероятно, что добавление биграмм, триграмм и т.д. становится осмысленным на существенно больших датасетах..


In [15]:
columns = ['Значение F1 обученной модели', 'Время обучения модели, мин']
resdata = [
    [0.752, 0.2],
    [0.652, 1.4],
    [0.766, 8.5]]

lists = ['Логистическая регрессия','Случайный лес','CatBoost']

resshow = pd.DataFrame(data = resdata, columns = columns, index = lists)

## Выводы

Для построения моделей использован датасет с комментариями относительно товаров интернет-магазина "Викишоп", всего приведено более 150 тысяч комментариев с разметкой о их "токсичности".

Для выполнения проекта выполнены следующие этапы:
- Очистка комментариев от системных сообщений, даты, названий вложенных изображений, смайликов и других "помех"
- Лемматизация текста с помощью библиотеки NLTK
- Векторизация комментариев для создания признаков с помощью TF-IDF

Были испробованы 3 модели, логистическая регрессия, случайный лес и CatBoost. Достигнутые значения метрики F1 на тестовой выборке и время обучения моделей приведены в таблице ниже:

In [16]:
display(resshow)

,Значение F1 обученной модели,"Время обучения модели, мин"
Логистическая регрессия,0.752,0.2
Случайный лес,0.652,1.4
CatBoost,0.766,8.5


Как можно заметить, наибольшее значение метрики было получено с помощью CatBoost, а быстрее всех обучилась логистическая регрессия. По критерию качество модели к времени обучения лидером стоит отметить именно регрессию.

Также на уменьшенном датасете была проверена гипотеза о том, уkучшится ли качество модели, если добавлять в качестве признаков не только униграммы, но и более высокие порядки. Отметим, что на уменьшенном датасете добавление лишних признаков только ухудшило знаечние метрики F1.